In [1]:
import requests
import pandas as pd
import numpy as np
import pebble_pb2

In [2]:
def run_query(q):
    request = requests.post('http://34.146.117.200:8000/subgraphs/name/iotex/pebble-subgraph'
                            '',
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed with return code: {}'.format(request.status_code))

In [3]:
query = """
{
  devices {
    id
    name
    address
    owner
    lastDataTime
    status
    name
    data
    config
  }
}
"""
result = run_query(query)

statusDf = pd.DataFrame(columns = ["Id", "Name", "Address", "Owner", "Last Data Time", "Status", "Name", "Raw Data",
                                  "Snr", "Vbat", "Latitude", "Longitude", "Gas Resistance", "Temperature", "Pressure",
                                  "Humidity", "Light", "Temperature2", "Gyroscope1", "Gyroscope2", "Gyroscope3",
                                  "Accelerometer1", "Accelerometer2", "Accelerometer3"])

for key, value in result["data"].items():
    for device in value:
        data = []
        data.append(device["id"])
        data.append(device["name"])
        data.append(device["address"])
        data.append(device["owner"])
        data.append(device["lastDataTime"])
        data.append(device["status"])
        data.append(device["name"])
        
        if(device["data"]):
            dataString = device["data"].replace("0x", "")
            data.append(dataString)
            dataBytes = bytes.fromhex(dataString)
            sensorData = pebble_pb2.SensorData()
            sensorData.ParseFromString(dataBytes)
            
            data.append(sensorData.snr)
            data.append(sensorData.vbat)
            data.append(sensorData.latitude)
            data.append(sensorData.longitude)
            data.append(sensorData.gasResistance)
            data.append(sensorData.temperature)
            data.append(sensorData.pressure)
            data.append(sensorData.humidity)
            data.append(sensorData.light)
            data.append(sensorData.temperature2)
            if(len(sensorData.gyroscope) == 3):
                data.append(sensorData.gyroscope[0])
                data.append(sensorData.gyroscope[1])
                data.append(sensorData.gyroscope[2])
            else:
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
            if(len(sensorData.accelerometer) == 3):
                data.append(sensorData.accelerometer[0])
                data.append(sensorData.accelerometer[1])
                data.append(sensorData.accelerometer[2])
            else:
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
        else:
            data.append("No Data")
            
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            
        statusDf.loc[len(statusDf)] = data

In [4]:
statusDf

,Id,Name,Address,Owner,Last Data Time,Status,Name,Raw Data,Snr,Vbat,...,Pressure,Humidity,Light,Temperature2,Gyroscope1,Gyroscope2,Gyroscope3,Accelerometer1,Accelerometer2,Accelerometer3
0,100000000000003,Pebble1,0xe1db026e0e8a359b7374a53ba50725dc6fc812e7,0x43bf16f19c10d95b35df9dc9ec3c74840472c4b7,1631631971,2,Pebble1,08cc0810a20118d894ddb70120a0a2afdd04288cfa2138...,1100,162,...,11915,7213,166950,3818,6,-5,-4021,2115,4304,4021
1,100000000000005,My Pebble,0xdab93b093cbb1ba91dd05add2ac25a0a2bb1d4b8,0x259c0a4251ee7cd3cba0a437973443c9c7cd2d4f,1631726165,2,My Pebble,082710201828200e2826301e38164026481f5025583c58...,39,32,...,22,38,31,37,30,25,29,28,37,29
2,100000000000006,Pebble Device 1,0x8ac9ba206f12327b30c377d5d63d4a304c6e3549,0x7ed220047b9a344e4868fc234d4d9c550a808d92,None,2,Pebble Device 1,No Data,No Data,No Data,...,No Data,No Data,No Data,No Data,No Data,No Data,No Data,No Data,No Data,No Data
3,100000000000007,Pebble1,0xcfe07c9e1bfa8915f3060577d72932df246831fb,0xd36eb6c726949264805e3f54e759f47e64073992,1631758747,2,Pebble1,08171015182720b5ffffff0f281d302138174022481850...,23,21,...,23,34,24,25,34,29,38,21,29,28
4,100000000000008,test,0x1973758805dc7690b8fb36b02854c2301cfc60bb,0x1973758805dc7690b8fb36b02854c2301cfc60bb,1631716346,2,test,081a101b182820b6ffffff0f2815301538244024482650...,26,27,...,36,36,38,37,21,36,32,32,32,37
5,100000000000009,my_pebble,0x843210f6b3333a082a90d9d79f23ddf98ac02964,0xbdcf0b9759c37dbcb37baaf43a7d27c5b74b9059,1631868581,2,my_pebble,08201016182720aeffffff0f281e301438244020481750...,32,22,...,36,32,23,22,39,30,35,29,33,36
6,100000000000093,100000000000093,0x153080670dbe11cffe8100ba5197b521c5bbd83b,0xf1c84e4c02407f24829e830d345473f748b09ca1,1631799487,2,100000000000093,0827102518b2cf80c20120f2b4a29f0d281a3022381c40...,39,37,...,28,27,24,24,36,39,38,37,38,26
7,100000000000094,100000000000094,0xfc71aef9cdc9f4815f6096a282a39fb56176cc28,0xf1c84e4c02407f24829e830d345473f748b09ca1,1631864425,2,100000000000094,081f1019182820b7ffffff0f2818301538224020481550...,31,25,...,34,32,21,22,37,24,36,27,38,28
8,100000000000096,My Pebble 3,0x85c40bc21a7116debbd5b84e213368f9f69895cc,0x259c0a4251ee7cd3cba0a437973443c9c7cd2d4f,1631905873,2,My Pebble 3,081e101918a8ac87c30120f885ada1042815301f382540...,30,25,...,37,30,39,37,33,25,23,22,39,36
9,100000000000097,100000000000097,0xffdae131f5d4a6364bccd0b4e213c352ecd2a4d7,0xe02fb9ad7b63c283847c418d3f34f9317b340d89,1631861942,2,100000000000097,0823101618b2cf80c20120f2b4a29f0d28223025381c40...,35,22,...,28,36,25,27,25,30,27,38,28,36
